In [1]:
# %%capture
# !pip install git+https://github.com/huggingface/datasets.git
# !pip install rouge_score
# !pip install sentencepiece
# !pip install transformers
# !pip install bert_score
# !pip install seaborn

In [2]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,T5Model
)
from datasets import load_dataset
from transformers import T5Tokenizer
from datasets import load_metric
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random


from transformers import T5Tokenizer
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import datasets



In [3]:
RANDOM_SEED = 42
BEAM_SIZE = 4
DEVICE = "cpu"
MODEL_NAME = "google/t5-v1_1-base"
DATASET_NAME = "e2e_nlg"
MAX_LENGTH = 512
BATCH_SIZE = 20
SAVE_EVAL_STRATEGY = 'epoch'

loading the data and EDA

In [4]:
data = pd.read_csv('Women_Boohoo_UK_processed.csv')

In [5]:
data = data.dropna(subset=['desc'])
data = data.dropna(subset=['name'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800 entries, 0 to 1801
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        1800 non-null   object
 1   gender      1800 non-null   object
 2   attributes  1279 non-null   object
 3   colour      1800 non-null   object
 4   price       1800 non-null   object
 5   category    1800 non-null   object
 6   desc        1800 non-null   object
dtypes: object(7)
memory usage: 112.5+ KB


split training and testing

In [6]:

data['source'] = data['name']
data = data.drop(
    ['name', 'gender', 'attributes', 'colour', 'price', 'category'], axis=1)

In [7]:
data

,desc,source
0,A classic wardrobe staple which no clothing co...,High Waisted Disco Denim Shorts
1,If you’re going for a top-to-bottom wardrobe r...,Black Crew Neck Basic Cotton Tshirt
2,Hit refresh on your casual wardrobe with a ver...,Faux Leather Trucker Jacket
3,A seriously comfy addition to your new-season ...,Cropped Slouchy Hoodie
4,"A menswear classic with a feminine edge, add a...",Pinstripe Tie Waist Fitted Tailored Blazer
...,...,...
1797,Introducing your new fave top from our latest ...,Plus Bandage Hook And Eye Corset Top
1798,Introducing your new fave top from our latest ...,Beige Plus Oversized Basic Sweatshirt
1799,Swapping out your jeans for something comfier?...,Beige Plus Double Cargo Pocket Oversized Jogger
1800,Just chilling? Do it right with an oversized h...,Dsgn Studio Slogan Oversized Hoodie


In [8]:
data.head()

,desc,source
0,A classic wardrobe staple which no clothing co...,High Waisted Disco Denim Shorts
1,If you’re going for a top-to-bottom wardrobe r...,Black Crew Neck Basic Cotton Tshirt
2,Hit refresh on your casual wardrobe with a ver...,Faux Leather Trucker Jacket
3,A seriously comfy addition to your new-season ...,Cropped Slouchy Hoodie
4,"A menswear classic with a feminine edge, add a...",Pinstripe Tie Waist Fitted Tailored Blazer


In [9]:
df_train = data.iloc[:100]   # First two rows of the dataframe
df_test = data.iloc[1750:]   # Remaining rows of the dataframe

In [10]:
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
    })

In [11]:
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['desc', 'source', '__index_level_0__'],
        num_rows: 2
    })
    test: Dataset({
        features: ['desc', 'source', '__index_level_0__'],
        num_rows: 50
    })
})

In [12]:
print(datasets_train_test['train'][0]['source'])

High Waisted Disco Denim Shorts


In [13]:
def construct_input_for_batch(batch):
    """Construct input strings from a batch."""
#     print(len(batch["source"]))
    source = batch["source"]
    target = batch["desc"]
    return source, target

In [14]:
def batch_tokenize(batch, tokenizer, max_length=32):
    """Construct the batch (source, target) and run them through a tokenizer."""
    source, target = construct_input_for_batch(batch)
    target_tokenized = tokenizer(
            batch['desc'],
            padding="max_length",
            truncation=True,
            max_length=max_length
        )
    target_input_ids = target_tokenized['input_ids']
    
    res = {
        "input_ids": tokenizer(source,
            padding="max_length",
            truncation=True,
            max_length=max_length)["input_ids"],
        "labels": target_input_ids,
    }
    return res

In [15]:

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

train_data_tokenized = datasets_train_test['train'].map(
    lambda batch: batch_tokenize(batch, tokenizer, max_length=MAX_LENGTH),
    batched=True
)
valid_data_tokenized = datasets_train_test['test'].map(
    lambda batch: batch_tokenize(batch, tokenizer, max_length=MAX_LENGTH),
    batched=True
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [16]:
meteor_scorer = load_metric('meteor')

def meteor_metric_builder(tokenizer):
    def compute_meteor_metrics(pred):
        """Utility to compute meteor during training."""
        labels_ids = pred.label_ids
        pred_ids = pred.predictions
        # All special tokens are removed.
        pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        labels_ids[labels_ids == -100] = tokenizer.pad_token_id
        label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
        # Compute the metric.
        meteor_results = meteor_scorer.compute(predictions=pred_str,
                                       references=label_str)
        return {
            "meteor": round(meteor_results['meteor'], 4),
        }
    return compute_meteor_metrics

meteor_metric_fn = meteor_metric_builder(tokenizer)

/var/folders/c7/9svjb46d4l14sh2yf88h65bh0000gn/T/ipykernel_82043/1771409562.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor_scorer = load_metric('meteor')
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhoutuanjie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhoutuanjie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zhoutuanjie/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    DEVICE = torch.ones(1, device=mps_device)
    print(DEVICE)
elif torch.cuda.is_available():
    DEVICE = "cuda:0"
    print(DEVICE)
    
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
model = model.to(DEVICE)

In [18]:
train_args = Seq2SeqTrainingArguments(
    output_dir="t5-v1_1-base-E2E",
    evaluation_strategy=SAVE_EVAL_STRATEGY,
    save_strategy=SAVE_EVAL_STRATEGY,
    logging_steps=5,
    # optimization args, the trainer uses the Adam optimizer
    # and has a linear warmup for the learning rate
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=1e-03,
    num_train_epochs=8,
    warmup_steps=1000,
    # misc args
    seed=RANDOM_SEED,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="meteor",
    # generation
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=train_data_tokenized,
    eval_dataset=valid_data_tokenized,
    tokenizer=tokenizer,
    compute_metrics=meteor_metric_fn,
)

trainer._max_length = MAX_LENGTH
trainer._num_beams = BEAM_SIZE

In [19]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, desc, source. If __index_level_0__, desc, source are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/zhoutuanjie/opt/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 8
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 8
  Number of trainable parameters = 247577856


Epoch,Training Loss,Validation Loss,Meteor
1,No log,50.294140,0.011400


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, desc, source. If __index_level_0__, desc, source are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 20
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Saving model checkpoint to t5-v1_1-base-E2E/checkpoint-1
Configuration saved in t5-v1_1-base-E2E/checkpoint-1/config.json
Configuration saved in t5-v1_1-base-E2E/checkpoint-1/generation_c

KeyboardInterrupt: 

In [ ]:
def beam_generate_sentences(batch,
                            model,
                            tokenizer,
                            num_beams=4,
                            max_length=128,
                            device='cuda:0'):
    """Generate outputs from a model with beam search decoding."""
    # Create batch inputs.
    source, _ = construct_input_for_batch(batch)
    # Use the model's tokenizer to create the batch input_ids.
    batch_features = tokenizer(source, padding=True, return_tensors='pt')
    # Move all inputs to the device.
    batch_features = dict([(k, v.to(device))
                           for k, v in batch_features.items()])

    # Generate with beam search.
    generated_ids = model.generate(
        **batch_features,
        num_beams=num_beams,
        max_length=max_length,
    )

    # Use model tokenizer to decode to text.
    generated_sentences = [
        tokenizer.decode(gen_ids.tolist(), skip_special_tokens=True)
        for gen_ids in generated_ids
    ]
    return generated_sentences

In [ ]:
valid_output = [X_test,y_test].map(
    lambda batch: {
        'generated':
        beam_generate_sentences(batch,
                                model,
                                tokenizer,
                                num_beams=BEAM_SIZE,
                                max_length=MAX_LENGTH,
                                device=DEVICE)
    },
    batched=True,
    batch_size=BATCH_SIZE,
)

In [ ]:
# Evaluate for ROUGE-2/L
rouge_scorer = load_metric("rouge")

rouge_results = rouge_scorer.compute(
    predictions=valid_output["generated"],
    references=valid_output["human_reference"],
    rouge_types=["rougeL"],
    use_aggregator=True,
    use_stemmer=False,
)
rougeL = rouge_results['rougeL'].mid.fmeasure
f"R-L: {rouge_results['rougeL'].mid.fmeasure:.3f}"

In [ ]:
rouge_results

In [ ]:
# Evaluate for meteor

meteor_results = meteor_scorer.compute(predictions=valid_output["generated"],
                                       references=valid_output["human_reference"])
meteor = meteor_results['meteor']
meteor_results

In [ ]:
bertscore = load_metric("bertscore")
bertscore_results = bertscore.compute(predictions=valid_output["generated"],
                                      references=valid_output["human_reference"],
                                      model_type='distilbert-base-uncased')

In [ ]:
bertscore_results

In [ ]:
def average(lst):
    return sum(lst) / len(lst)

bert_average_precision = average(bertscore_results['precision'])
bert_average_recall = average(bertscore_results['recall'])
bert_average_f1 = average(bertscore_results['f1'])

f'average_precision: {bert_average_precision}, average_recall: {bert_average_recall},average_f1: {bert_average_f1}'

In [ ]:
bert_score_merics = [
    "rougeL", "meteor", "bert_average_precision", "bert_average_recall",
    "bert_average_f1"
]
bert_score_list = [
    rougeL, meteor, bert_average_precision, bert_average_recall,
    bert_average_f1
]

dataf = pd.DataFrame({
    "bert_score_merics": bert_score_merics,
    "bert_score_list": bert_score_list
})

plt.figure(figsize=(12, 6), dpi=80)
sns.barplot(x="bert_score_merics",
            y="bert_score_list",
            data=dataf,
            palette='Blues')
plt.show()

In [ ]:
random.choice(valid_output)